In [14]:
import pandas as pd
import spacy
from sklearn.metrics import make_scorer, classification_report, make_scorer, recall_score, f1_score, accuracy_score, confusion_matrix
import time
import seaborn as sns
import matplotlib.pyplot as plt


train = pd.read_csv("../data/avis/train_noYC_lemma_sent_equil.csv", index_col= 0)
df = pd.read_excel('../data/avis/general_df_clean_sent_15k_manual_code.xlsx')

nlp = spacy.load("fr_core_news_lg")



In [12]:
theme_frais= [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_frais'])]
theme_efficacite = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_efficacité'])]
theme_discrimination = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_discrimination'])]
theme_protexction = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_protection'])]
theme_cloture = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_clôture'])]
theme_prets = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_prêts'])]
theme_virements = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_virements'])]
theme_communication = [row['text_avis'] for index, row in df.iterrows() if pd.notna(row['var_communication'])]

nuggets_efficacité = pd.read_excel("../data/avis/general_df_clean_sent_15k_manual_code.xlsx",
                                   sheet_name = "curation_efficacité", index_col= 0)


# efficace 
list_exemples_efficace = nuggets_efficacité.curated_efficace
list_exemples_efficace = list_exemples_efficace.dropna()
bag_efficace = " ".join([message for message in list_exemples_efficace])

# innefficace
list_exemples_innefficace = nuggets_efficacité.curated_inneficace
list_exemples_innefficace = list_exemples_innefficace.dropna()
bag_innefficace = " ".join([message for message in list_exemples_innefficace])
bag_innefficace_nlp =nlp(bag_innefficace)

nuggets_efficacité.head(1)

,raw_efficacité_efficace,curated_efficace,raw_ineficace,curated_inneficace
NaN,"Je suis satisfaite de l'accueil,du suivi merc...","Je suis satisfaite de l'accueil,du suivi merc...",délai trop long entre dossier et réception arg...,délai trop long entre dossier et réception arg...


In [58]:
mini = df.iloc[0:50,:]
train.head(1)

,Unnamed: 0,user,etoiles,n_avis,date_avis,date_experience,Société,text_total,jour_avis,mois_avis,...,heure_experience,annee_experience,periode_avis,longueur_texte,text_stop,text_lemma,label,score,sentiment_norm,var_efficacité_bad
5529,5529,Benjamin Danneville,5,1.0,2021-04-26,2021-04-26,Shine,Permet de faire des factures et des…Permet de ...,26,4,...,0,2021,nuit,95,Permet faire factures … Permet faire factures ...,permettre faire facture … permettre faire fact...,1,0.804014,0.804014,0


In [56]:
def label_banks(df,ref_bag,lab_col,thresh = 0.83):
    y_pred = []
    round_count = 0
    t0 = time.time() # initialising empty variables and the timer
    for message in df.text_total: # for each review
        round_count+=1
        sim_score= []
        message = nlp(message)
        for unlabeled_sentence in message.sents: # for every sentence in the review
            for labeled_sentence in ref_bag.sents : # for every sentence in our reference bag of labeled reviews
                temp_sim_score = round(labeled_sentence.similarity(unlabeled_sentence),2) # check similarity between review sentence <> reference sentence
                sim_score.append(temp_sim_score)
        if max(sim_score) > thresh:
            y_pred.append(1)
        else:
            y_pred.append(0)
        if round_count % 100 == 0: # notification every 100 passed review
            print(round_count, " messages labelled")
    t1 = time.time() # stopping the timer
    delais = round((t1-t0)/60,2)
    df[lab_col] = y_pred # integrating result in our source dataframe
    print(f"{delais} minutes to compete similarity labelling")
    df[lab_col].value_counts()

In [57]:
y_pred = []
label_banks(train,bag_innefficace_nlp,"var_efficacité_bad",0.83)

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_45500/3635321684.py:11: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  temp_sim_score = round(labeled_sentence.similarity(unlabeled_sentence),2) # check similarity between review sentence <> reference sentence


100  messages labelled
200  messages labelled
300  messages labelled
400  messages labelled
500  messages labelled
600  messages labelled
700  messages labelled
800  messages labelled
900  messages labelled
1000  messages labelled
1100  messages labelled
1200  messages labelled
1300  messages labelled
1400  messages labelled
1500  messages labelled
1600  messages labelled
1700  messages labelled
1800  messages labelled
1900  messages labelled
2000  messages labelled
2100  messages labelled
2200  messages labelled
2300  messages labelled
2400  messages labelled
2500  messages labelled
2600  messages labelled
2700  messages labelled
2800  messages labelled
2900  messages labelled
3000  messages labelled
3100  messages labelled
3200  messages labelled
3300  messages labelled
3400  messages labelled
3500  messages labelled
3600  messages labelled
3700  messages labelled
3800  messages labelled
3900  messages labelled
4000  messages labelled
4100  messages labelled
4200  messages labelled
4

In [50]:
mini

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,codes agrégés,comment,var_frais,var_efficacité,var_efficacité_bad,var_efficacité_good,var_discrimination,var_protection,var_clôture,var_prêts,var_virements,var_communication
0,Sylvain Andujar,3,3,2022-07-26 10:10:19,Pour l'instant toujours en attente,"Pour l'instant toujours en attente, tout les j...",bad:inefficace,NaN,NaN,bad:inefficace,1,0,NaN,NaN,NaN,NaN,NaN,NaN
1,ANNICK JALLET,3,1,2022-04-11 14:14:30,intérêts trop élevés,na,na,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,Client,2,3,2023-06-07 16:57:39,"C'était les meilleurs, mais c'était avant",Deux dernières fois où j'ai dû faire appel à F...,bad:communication; bad:ineficace,NaN,NaN,bad:inefficace,1,0,NaN,NaN,NaN,NaN,NaN,bad:communication
3,Radio Anime,1,1,2023-09-08 13:30:04,Vous n'aidez pas les gens,"Vous n'aidez pas les gens, vous faites l'inver...",na,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Serge Péon,1,2,2023-02-10 20:32:37,De voyage à l'étranger mon conseillé de…,De voyage à l'étranger mon conseillé de la Ban...,bad:clôture_abusive,NaN,NaN,NaN,1,0,NaN,NaN,bad:clôture_abusive,NaN,NaN,NaN
5,Juliana VAX,5,3,2023-10-19 10:14:14,Simple et efficace,"Le service est très simple, rapide et efficace...",good:communication,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,good:communication
6,Perez Vicencia,5,1,2023-10-17 11:24:02,Très efficace,"Très efficace,rapide et sécurisé. J'ai reçu un...",good:communication; good:prêt_accordé,NaN,NaN,NaN,1,0,NaN,NaN,NaN,good:prêt_accordé,NaN,good:communication
7,client,3,1,2018-10-03 06:25:45,Procédure en ligne à revoir,Procédure en ligne à revoir. Si il manque des ...,bad:communication,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,bad:communication
8,Philippe,4,3,2017-11-06 13:48:15,formulaire clair et précis,"formulaire clair et précis, traitement rapide ...",good:communication,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,good:communication
9,MLE MARIE HELENE C,3,1,2022-02-22 10:14:31,"Je suis satisfaite de l'accueil,du…","Je suis satisfaite de l'accueil,du suivi merc...",good:communication; good:efficace,NaN,NaN,good:efficace,0,0,NaN,NaN,NaN,NaN,NaN,good:communication


In [ ]:
# top_bank topbank
list_top = [bank for bank in df.Société.value_counts().keys()[0:10]]
df_top = df[df['Société'].isin(list_top)]

import seaborn as sns

sns.countplot(x = df_top['var_bad:inefficace']== 1, hue = df_top.Société)